In [1]:
chrome_driver = './driver/chromedriver.exe'

In [2]:
URL = "https://play.google.com/store/apps/details?id=com.trenbe.trenbehybrid&hl=ko-KR"

In [3]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd

In [4]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

In [5]:
# 크롬 드라이버 세팅
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(chrome_driver, options = chrome_options)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
all_review_button_xpath = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[2]/div[2]/div/div[1]/div[1]/c-wiz[4]/section/div/div[2]/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element(By.XPATH,all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

C:\Users\hsty9\AppData\Local\Temp\ipykernel_11288\1432366863.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver, options = chrome_options)


스크롤 완료


In [6]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [7]:
driver.quit()

In [8]:
# html 데이터 저장
with open("../data/t_data_html.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

In [9]:
from tqdm import tqdm

In [10]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

100%|█████████████████████████████████████████████████████████████████████████████| 3960/3960 [00:17<00:00, 220.42it/s]


In [11]:
df = pd.DataFrame(dataset)
df.to_csv('../data/t_review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

In [12]:
df = pd.read_csv('../data/t_review_dataset.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20211026,2021,10,26,1,망이,"앱 오류를 고치던지 관리 좀 하세요 1. 앱 접속시 로딩 개느림 -> 와이파이, 데..."
1,2,20220928,2022,9,28,5,홀릭블릭,베타이후가 확실히 편하고 좋아요! 그런데 알림설정은 어디서 하는지 잘 모르겠네요.....
2,3,20220928,2022,9,28,5,채수하,기능과 아이디어는 패션 앱중 정말 혁신적이고 되게 좋습니다! 하지만 보완해야 할 점...
3,4,20220928,2022,9,28,5,나무별,"혜택도 좋은 것 같고, 쿠폰도 구간별로 잘 지원해주는 것 같으며, 고품질의 질 좋은..."
4,5,20220928,2022,9,28,5,rose young,편의성 있고 아주 유용하게 사용하고 있습니다. 화면도 깔끔하고 단순해서 거래하기 쉬...
...,...,...,...,...,...,...,...,...
3955,3956,20211223,2021,12,23,1,김삼영,앱안깔아야지.
3956,3957,20210827,2021,8,27,5,유지혜,좋아요
3957,3958,20211102,2021,11,2,5,심수하,WOW~그레잇~~~~🐣
3958,3959,20211014,2021,10,14,5,paper moon,Good!! Nice!!


In [13]:
df.sort_values(by='date', ascending=False, inplace=True)
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
3269,3270,20230912,2023,9,12,1,처로로,"욕이 아까울정도로 무책임,무성의,무대응 고객을 아주 x으로 아는곳"
43,44,20230911,2023,9,11,1,안안현주,예전부터 앱 잘쓰고 있었는데 요새들어 로딩이 느려지고 렉도 잘걸려서 화면이 정지가 ...
45,46,20230906,2023,9,6,2,SEONHWAN CHOI,앱이 진짜 너무 느립니다 서버를 확충해야 되는 지 뭔지 모르겠지만 이용할 때마다 답...
173,174,20230902,2023,9,2,1,씨니,"아오 렉이 오지게 걸려서 쇼핑을 할수가없네요ㅡㅡ 앱을 삭제하던가해야지 원,"
54,55,20230826,2023,8,26,2,RITARITA,스크롤 안 내려가는 오류는 대체 언제 고쳐질까요
...,...,...,...,...,...,...,...,...
3861,3862,20190720,2019,7,20,5,Google 사용자,좋아요!!!!!!!!
3766,3767,20190719,2019,7,19,5,Google 사용자,좋아요 백점만점~~
2289,2290,20190719,2019,7,19,5,Google 사용자,트렌비 자주 보는데... 맨날 크롬 띄워서 봤는데... 이젠 앱만 클릭하면 나오니 ...
3828,3829,20190719,2019,7,19,5,Google 사용자,깔끔하고 좋아요


In [14]:
df.reset_index(drop=True, inplace=True)
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,3270,20230912,2023,9,12,1,처로로,"욕이 아까울정도로 무책임,무성의,무대응 고객을 아주 x으로 아는곳"
1,44,20230911,2023,9,11,1,안안현주,예전부터 앱 잘쓰고 있었는데 요새들어 로딩이 느려지고 렉도 잘걸려서 화면이 정지가 ...
2,46,20230906,2023,9,6,2,SEONHWAN CHOI,앱이 진짜 너무 느립니다 서버를 확충해야 되는 지 뭔지 모르겠지만 이용할 때마다 답...
3,174,20230902,2023,9,2,1,씨니,"아오 렉이 오지게 걸려서 쇼핑을 할수가없네요ㅡㅡ 앱을 삭제하던가해야지 원,"
4,55,20230826,2023,8,26,2,RITARITA,스크롤 안 내려가는 오류는 대체 언제 고쳐질까요
...,...,...,...,...,...,...,...,...
3955,3862,20190720,2019,7,20,5,Google 사용자,좋아요!!!!!!!!
3956,3767,20190719,2019,7,19,5,Google 사용자,좋아요 백점만점~~
3957,2290,20190719,2019,7,19,5,Google 사용자,트렌비 자주 보는데... 맨날 크롬 띄워서 봤는데... 이젠 앱만 클릭하면 나오니 ...
3958,3829,20190719,2019,7,19,5,Google 사용자,깔끔하고 좋아요


In [15]:
df.drop(columns='id', inplace=True)
df

,date,dateYear,dateMonth,dateDay,rating,userName,content
0,20230912,2023,9,12,1,처로로,"욕이 아까울정도로 무책임,무성의,무대응 고객을 아주 x으로 아는곳"
1,20230911,2023,9,11,1,안안현주,예전부터 앱 잘쓰고 있었는데 요새들어 로딩이 느려지고 렉도 잘걸려서 화면이 정지가 ...
2,20230906,2023,9,6,2,SEONHWAN CHOI,앱이 진짜 너무 느립니다 서버를 확충해야 되는 지 뭔지 모르겠지만 이용할 때마다 답...
3,20230902,2023,9,2,1,씨니,"아오 렉이 오지게 걸려서 쇼핑을 할수가없네요ㅡㅡ 앱을 삭제하던가해야지 원,"
4,20230826,2023,8,26,2,RITARITA,스크롤 안 내려가는 오류는 대체 언제 고쳐질까요
...,...,...,...,...,...,...,...
3955,20190720,2019,7,20,5,Google 사용자,좋아요!!!!!!!!
3956,20190719,2019,7,19,5,Google 사용자,좋아요 백점만점~~
3957,20190719,2019,7,19,5,Google 사용자,트렌비 자주 보는데... 맨날 크롬 띄워서 봤는데... 이젠 앱만 클릭하면 나오니 ...
3958,20190719,2019,7,19,5,Google 사용자,깔끔하고 좋아요


In [16]:
df.to_csv('../data/trenbe_google.csv', encoding = 'utf-8-sig')